In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 500     # number of variables
num_ineq = 500    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_500-500.csv")

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 16%|███████████▉                                                                 | 155/1000 [15:18<1:34:38,  6.72s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 18%|█████████████▌                                                               | 176/1000 [17:19<1:19:02,  5.76s/it]

...

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 89%|████████████████████████████████████████████████████████████████████▊        | 893/1000 [1:28:11<10:20,  5.80s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:38:31<00:00,  5.91s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean   -106.514718         66.072066      1.276911
std       0.477966          5.158179      0.213347
min    -108.255859         52.381631      1.068142
25%    -106.823279         62.708049      1.124031
50%    -106.520729         65.936749      1.165816
75%    -106.202046         69.647282      1.384423
max    -104.765442         86.801004      2.259209
Number of infeasible solution: 1000


In [7]:
df = pd.read_csv("result/cq_heur_rnd_500-500.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean  -106.526320         65.849008      1.458994
std      0.498114          5.384539      0.281285
min   -107.598198         54.856843      1.073386
25%   -106.903252         62.310163      1.177580
50%   -106.536356         65.966554      1.493044
75%   -106.184064         69.026879      1.655169
max   -105.364271         81.729129      2.259209
Number of infeasible solution: 100


## Learnable Rounding

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 34831.11
Epoch 1, Validation Loss: 31.45
Epoch 2, Validation Loss: -2.14
Epoch 3, Validation Loss: -15.80
Epoch 4, Validation Loss: -26.56
Epoch 5, Validation Loss: -34.21
Epoch 6, Validation Loss: -40.77
Epoch 7, Validation Loss: -42.90
Epoch 8, Validation Loss: -34.05
Epoch 9, Validation Loss: -46.91
Epoch 10, Validation Loss: -43.30
Epoch 11, Validation Loss: -47.84
Epoch 12, Validation Loss: -49.80
Epoch 13, Validation Loss: -54.40
Epoch 14, Validation Loss: -52.03
Epoch 15, Validation Loss: -51.60
Epoch 16, Validation Loss: -54.75
Epoch 17, Validation Loss: -54.21
Epoch 18, Validation Loss: -57.02
Epoch 19, Validation Loss: -57.37
Epoch 20, Validation Loss: -57.39
Epoch 21, Validation Loss: -56.11
Epoch 22, Validation Loss: -58.45
Epoch 23, Validation Loss: -47.94
Epoch 24, Validation Loss: -56.49
Epoch 25, Validation Loss: -60.88
Epoch 26, Validation Loss: -60.82
Epoch 27, Validation Loss: -58.35
Epoch 28, Validation Loss: -61.94
Epoch 29, Validation Los

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_500-500.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [11:33<00:00,  1.44it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -73.656593          0.007516      0.011249
std       0.588918          0.052571      0.025803
min     -74.352417          0.000000      0.002002
25%     -73.856827          0.000000      0.003652
50%     -73.792507          0.000000      0.004562
75%     -73.610165          0.000000      0.007214
max     -67.608421          0.798371      0.256757
Number of infeasible solution: 46


In [8]:
df = pd.read_csv("result/cq_lr_500-500.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -73.569562          0.012426      0.007417
std      0.733295          0.070287      0.010656
min    -74.352417          0.000000      0.002459
25%    -73.837213          0.000000      0.003927
50%    -73.761501          0.000000      0.004852
75%    -73.591044          0.000000      0.006606
max    -70.282717          0.643432      0.098346
Number of infeasible solution: 8


## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12499.07
Epoch 1, Validation Loss: 23.34
Epoch 2, Validation Loss: 5.01
Epoch 3, Validation Loss: -13.72
Epoch 4, Validation Loss: -25.57
Epoch 5, Validation Loss: -34.12
Epoch 6, Validation Loss: -34.23
Epoch 7, Validation Loss: -40.39
Epoch 8, Validation Loss: -45.40
Epoch 9, Validation Loss: -45.98
Epoch 10, Validation Loss: -38.56
Epoch 11, Validation Loss: -49.05
Epoch 12, Validation Loss: -51.56
Epoch 13, Validation Loss: -48.03
Epoch 14, Validation Loss: -51.90
Epoch 15, Validation Loss: -55.42
Epoch 16, Validation Loss: -53.27
Epoch 17, Validation Loss: -58.66
Epoch 18, Validation Loss: -58.44
Epoch 19, Validation Loss: -57.09
Epoch 20, Validation Loss: -52.19
Epoch 21, Validation Loss: -57.58
Epoch 22, Validation Loss: -58.68
Epoch 23, Validation Loss: -57.29
Epoch 24, Validation Loss: -43.66
Epoch 25, Validation Loss: -59.77
Epoch 26, Validation Loss: -59.96
Epoch 27, Validation Loss: -61.38
Epoch 28, Validation Loss: -61.83
Epoch 29, Validation Loss

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_500-500.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [11:29<00:00,  1.45it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -73.476697          0.010866      0.010322
std       0.385276          0.052991      0.023218
min     -74.087269          0.000000      0.002002
25%     -73.769244          0.000000      0.003704
50%     -73.515358          0.000000      0.004543
75%     -73.272909          0.000000      0.007407
max     -70.817713          0.658992      0.225816
Number of infeasible solution: 68


In [9]:
df = pd.read_csv("result/cq_lt_500-500.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -73.456509          0.017393      0.009755
std      0.365448          0.063938      0.023271
min    -74.013422          0.000000      0.002513
25%    -73.758096          0.000000      0.003572
50%    -73.413599          0.000000      0.004235
75%    -73.232405          0.000000      0.007030
max    -71.882978          0.332926      0.170428
Number of infeasible solution: 8


## Parametric Learning Then Rounding¶

In [10]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [12]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [13]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12553.04
Epoch 1, Validation Loss: -1.33
Epoch 2, Validation Loss: -16.12
Epoch 3, Validation Loss: -29.17
Epoch 4, Validation Loss: -37.21
Epoch 5, Validation Loss: -43.48
Epoch 6, Validation Loss: -47.20
Epoch 7, Validation Loss: -50.64
Epoch 8, Validation Loss: -53.48
Epoch 9, Validation Loss: -56.61
Epoch 10, Validation Loss: -61.43
Epoch 11, Validation Loss: -61.15
Epoch 12, Validation Loss: -63.67
Epoch 13, Validation Loss: -65.55
Epoch 14, Validation Loss: -65.49
Epoch 15, Validation Loss: -67.17
Epoch 16, Validation Loss: -66.86
Epoch 17, Validation Loss: -69.99
Epoch 18, Validation Loss: -69.55
Epoch 19, Validation Loss: -70.48
Epoch 20, Validation Loss: -68.71
Epoch 21, Validation Loss: -70.86
Epoch 22, Validation Loss: -72.14
Epoch 23, Validation Loss: -73.24
Epoch 24, Validation Loss: -70.37
Epoch 25, Validation Loss: -74.17
Epoch 26, Validation Loss: -74.73
Epoch 27, Validation Loss: -75.11
Epoch 28, Validation Loss: -77.63
Epoch 29, Validation Lo

In [15]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_pr_500-500.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [11:11<00:00,  1.49it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -90.205391          4.366663      0.000813
std       0.001680          1.280261      0.000504
min     -90.258474          0.739293      0.000000
25%     -90.205338          3.510456      0.000505
50%     -90.205338          4.275835      0.001000
75%     -90.205338          5.230381      0.001009
max     -90.205338          8.286500      0.002515
Number of infeasible solution: 1000


In [16]:
df = pd.read_csv("result/cq_pr_500-500.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean  -9.020534e+01          4.344916      0.000843
std    1.285420e-13          1.152884      0.000493
min   -9.020534e+01          1.866629      0.000000
25%   -9.020534e+01          3.649780      0.000858
50%   -9.020534e+01          4.132823      0.001000
75%   -9.020534e+01          5.084770      0.001013
max   -9.020534e+01          8.286500      0.001869
Number of infeasible solution: 100
